In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pandas.plotting import scatter_matrix
from matplotlib import pyplot as plt
from sklearn import tree
#import pydotplus
import collections
from IPython.display import Image  
#import pydotplus
from sklearn.tree import export_graphviz

In [1]:
import pandas as pd
import numpy as np

In [2]:
import copy

In [2]:
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [3]:
df = pd.read_stata("Econ_484_data_final.dta")

In [4]:
# df.isin(["''"]).sum(axis=1).sum()

In [5]:
df = df.drop("DAYSWAIT_CHRON", axis=1)

In [6]:
df = df.dropna(axis=0, how='any')

In [7]:
df['year'] = pd.DatetimeIndex(df["TX_DATE"]).year
df['month'] = pd.DatetimeIndex(df["TX_DATE"]).month
df = df.drop("TX_DATE", axis=1)

### Howdie y'all. Here's the "Average person" code
I decided it made the most sense to do it here because this is before we turn our "object" (strings) into ints/floats. It shoul dbe easy to move between stuff from here but let me know if you have to think about anything for more than like 10 minutes because if so, you have to work harder than I intended.

It's pretty simple. Sorry I didn't get to it until just barely.

In [8]:
# considering everything together
take_mode = df.dtypes == "object"
df.loc[:,take_mode].mode()

,gender,abo,PERM_STATE,EXH_PERIT_ACCESS,EXH_VASC_ACCESS,PREV_TX,PREV_KI_TX,MALIG_TRR,txkid,ABO_DON,...,HOME_STATE_DON,ABO_MAT,GRF_STAT_KI,DWFG_KI,PREV_TX_ANY,PX_STAT,AGE_GROUP,malig,LT_ONE_WEEK_DON,RECOV_OUT_US
0,M,O,CA,N,N,N,N,N,L,O,...,CA,1,Y,N,N,A,A,N,N,N


In [9]:
df.loc[:,~take_mode].median()

NUM_PREV_TX             0.0
A1                      2.0
A2                     25.0
B1                     35.0
B2                     50.0
DR1                     7.0
DR2                    11.0
REM_CD                  4.0
END_STAT             4010.0
INIT_AGE               47.0
ethnicity               0.0
ethcat                  1.0
region                  5.0
RB1                    35.0
npkid                   0.0
AGE_DON                38.0
ETHCAT_DON              1.0
END_STAT_KI          4010.0
age                    49.0
GTIME_KI             1696.0
GSTATUS_KI              0.0
DAYSWAIT_CHRON_KI     378.0
TX_PROCEDUR_TY_KI     101.0
SHARE_TY                3.0
pstatus                 0.0
ptime                2041.0
year                 2007.0
month                   7.0
dtype: float64

In [10]:
df['groups'] = pd.qcut(df['DAYSWAIT_CHRON_KI'], [0,.2,.8,1],labels=['fast','medium','slow'])
take_mode = df.dtypes == "object" # we've got an extra column now

In [11]:
df.groupby('groups').apply(lambda x: x.mode()).loc[:,take_mode]

,,gender,abo,PERM_STATE,EXH_PERIT_ACCESS,EXH_VASC_ACCESS,PREV_TX,PREV_KI_TX,MALIG_TRR,txkid,ABO_DON,...,HOME_STATE_DON,ABO_MAT,GRF_STAT_KI,DWFG_KI,PREV_TX_ANY,PX_STAT,AGE_GROUP,malig,LT_ONE_WEEK_DON,RECOV_OUT_US
groups,,,,,,,,,,,,,,,,,,,,,,
fast,0,M,A,CA,N,N,N,N,N,L,O,...,TX,1,Y,N,N,A,A,N,N,N
medium,0,M,O,CA,N,N,N,N,N,L,O,...,CA,1,Y,N,N,A,A,N,N,N
slow,0,M,O,CA,N,N,N,N,N,L,O,...,CA,1,Y,N,N,A,A,N,N,N


In [12]:
ave_person_1 = df.groupby('groups').apply(lambda x: x.mode()).loc[:,take_mode]

In [13]:
data_ave_1 = ave_person_1.to_excel(r'C:\Users\jordan79\Desktop\ave_person_1.xlsx', index= None, header=True)



In [13]:
df.groupby('groups').apply(lambda x: x.mode()).loc[:,take_mode].nunique()
# Not super promising

gender              1
abo                 2
PERM_STATE          1
EXH_PERIT_ACCESS    1
EXH_VASC_ACCESS     1
PREV_TX             1
PREV_KI_TX          1
MALIG_TRR           1
txkid               1
ABO_DON             1
DON_TY              1
GENDER_DON          1
HOME_STATE_DON      2
ABO_MAT             1
GRF_STAT_KI         1
DWFG_KI             1
PREV_TX_ANY         1
PX_STAT             1
AGE_GROUP           1
malig               1
LT_ONE_WEEK_DON     1
RECOV_OUT_US        1
dtype: int64

In [14]:
df.groupby('groups').apply(lambda x: x.median()).loc[:,~take_mode]

,NUM_PREV_TX,A1,A2,B1,B2,DR1,DR2,REM_CD,END_STAT,INIT_AGE,...,age,GTIME_KI,GSTATUS_KI,DAYSWAIT_CHRON_KI,TX_PROCEDUR_TY_KI,SHARE_TY,pstatus,ptime,year,month
groups,,,,,,,,,,,,,,,,,,,,,
fast,0.0,2.0,24.0,27.0,49.0,4.0,11.0,4.0,4010.0,47.0,...,48.0,1643.0,0.0,45.0,101.0,3.0,0.0,2031.0,2006.0,7.0
medium,0.0,2.0,24.0,35.0,50.0,7.0,11.0,4.0,4010.0,48.0,...,49.0,1802.0,0.0,379.0,101.0,3.0,0.0,2177.0,2006.0,7.0
slow,0.0,2.0,28.0,35.0,51.0,7.0,12.0,4.0,4010.0,47.0,...,52.0,1443.0,0.0,1525.0,102.0,3.0,0.0,1677.0,2010.0,7.0


In [15]:
df.groupby('groups').apply(lambda x: x.median()).loc[:,~take_mode].nunique()
# doesn't look supe rdifferent to me...

NUM_PREV_TX          1
A1                   1
A2                   2
B1                   2
B2                   3
DR1                  2
DR2                  2
REM_CD               1
END_STAT             1
INIT_AGE             2
ethnicity            1
ethcat               2
region               1
RB1                  1
npkid                1
AGE_DON              3
ETHCAT_DON           1
END_STAT_KI          1
age                  3
GTIME_KI             3
GSTATUS_KI           1
DAYSWAIT_CHRON_KI    3
TX_PROCEDUR_TY_KI    2
SHARE_TY             1
pstatus              1
ptime                3
year                 2
month                1
dtype: int64

In [17]:

ave_person_2 = df.groupby('groups').apply(lambda x: x.median()).loc[:,~take_mode]

data_ave_2 = ave_person_2.to_excel(r'C:\Users\jordan79\Desktop\ave_person_2.xlsx', index= None, header=True)


In [16]:
dum_ave_3  = pd.get_dummies(ave_person_1)
data_ave_3 = dum_ave_3.to_excel(r'C:\Users\jordan79\Desktop\ave_person_3.xlsx', index= None, header=True)

### Making dummy variables

- gender
- abo{9}
- EXH_PERIT_ACCCESS{3}
- EXH_VASC_ACCESS{3}
- PREV_TX{2}
- PREV_KI_TX{2}
- MALIG_TRR{3}
- PERM_STATE_TRR{51}
- txkid{3}
- ABO_DON{9}
- DON_TY{3}
- GENDER_DON{2}
- HOME_STATE_DON{51}
- ABO_MAT{3}
- GRF_STAT_KI{2}
- DWFG_KI{2}
- TX_PROCEDUR_TY_KI{[did you get one or two? See data_sheet]}
- PRV_TX_ANY{2}
- PX_STAT{4}
- SHARE_TY{4}
- AGE_GROUP{2}
- malig{3} (previous malignancies)
- LT_ONE_WEEK_DON {2} (7 days old or less?)
- RECOV_OUT_US{2}

In [16]:
dummy_columns = ['gender', 'abo', 'EXH_PERIT_ACCESS', 'EXH_VASC_ACCESS', 'PREV_TX', 'PREV_KI_TX',
                 'MALIG_TRR', 'txkid', 'ABO_DON', 'DON_TY', 'GENDER_DON', 'DON_TY', 'GENDER_DON',
                 'HOME_STATE_DON', 'ABO_MAT', 'GRF_STAT_KI', 'DWFG_KI', 'PREV_TX_ANY', 'PX_STAT',
                 'SHARE_TY', 'AGE_GROUP', 'malig', 'LT_ONE_WEEK_DON', 'RECOV_OUT_US', 'year', 'month',
                 'PERM_STATE']
funky_columns = ['TX_PROCEDUR_TY_KI']

In [17]:
df_non_dummy = df.drop(dummy_columns, axis=1)
ave_person
#df_non_dummy

In [18]:
df_dummified = pd.get_dummies(df[dummy_columns])


In [19]:
df = df_non_dummy.merge(df_dummified, left_index=True, right_index=True)


In [20]:
df.columns

Index(['NUM_PREV_TX', 'A1', 'A2', 'B1', 'B2', 'DR1', 'DR2', 'REM_CD',
       'END_STAT', 'INIT_AGE',
       ...
       'PERM_STATE_TX', 'PERM_STATE_UT', 'PERM_STATE_VA', 'PERM_STATE_VI',
       'PERM_STATE_VT', 'PERM_STATE_WA', 'PERM_STATE_WI', 'PERM_STATE_WV',
       'PERM_STATE_WY', 'PERM_STATE_ZZ'],
      dtype='object', length=222)

In [21]:
y_variable = 'DAYSWAIT_CHRON_KI'
X_variables = list(df.columns[df.columns != y_variable])
X = df[X_variables].to_numpy()
y = df[y_variable].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y)

forest = RandomForestRegressor(n_estimators=5)
model = forest.fit(X_train, y_train)

ValueError: could not convert string to float: 'medium'

In [ ]:
model.score(X_test,y_test)

In [ ]:
def plot_feature_importances_(model):
    n_features = X_test.shape[1]
    plt.barh(np.arange(n_features),model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features),df.columns)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

plot_feature_importances_(model)

In [ ]:
num_21 =  X[21,:].copy().reshape(1, -1)
num_21.shape


In [ ]:
num_21  = pd.DataFrame(num_21, columns = df[X_variables].columns)
num_21

In [ ]:
num_21.iloc[0,221]

In [ ]:
num_21_different_state = copy.deepcopy(num_21)
num_21_different_state

In [ ]:

num_21_different_state.iloc[0,220]=0
num_21_different_state.iloc[0,180]=1
num_21_different_state

In [ ]:
num_21


In [ ]:
num_21_different_state


In [ ]:
num_21 = num_21.to_numpy()
num_21_different_state = num_21_different_state.to_numpy()

In [ ]:
print(model.predict(num_21))
print(model.predict(num_21_different_state))

In [ ]:
df[X_variables].columns

In [ ]:

#num_21 =  X[21,:172].copy().reshape(1, -1)
List = []
for i in range(58):
    A = np.zeros((58,), dtype=int)
    A[i]=1
    num_10000 = np.concatenate((X[349885,:172].copy(),A), axis = None).reshape(1,-1)
    List.append(model.predict(num_10000))
    print(model.predict(num_10000))
    

In [ ]:
state_names = ['AK','AL','AR','AS','AZ','CA','CO','CT','DC','DE','FL','GA',
               'GU','HI','IA','ID','IL','IN','KS','KY','LA','MA','MD','ME',
               'MI','MN','MO','MP','MS','MT','NA','NC','ND','NE','NH','NJ',
               'NM','NV','NY','OH','OK','OR','PA','PR','RI','SC','SD','TN',
               'TX','UT','VA','VI','VT','WA','WI','WV','WY','ZZ']


In [ ]:
List = np.asarray(List)
#List.reshape(-1,1)

wl_times = pd.DataFrame()
wl_times['state']= state_names
wl_times['days']= List


In [ ]:
List = np.asarray(List)
wl_times['days_4']=List
wl_times

In [ ]:
wl_times.describe()


In [ ]:
num_10000 = np.concatenate((X[10000,:179].copy(),np.zeros(51)), axis = None).reshape(1,-1)
num_10000.shape

In [ ]:
num_10000  = pd.DataFrame(num_10000, columns = df[X_variables].columns)
num_10000
